## XGBデフォルト

In [1]:
import numpy as np
import pandas as pd

import xgboost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import train_test_split


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk3')

In [3]:
train_pkl.shape

(55323, 12)

## 訓練データとテストデータに分割

In [4]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [5]:
# ターゲットと特徴量の分割
train_X_tmp = train.copy()
train_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
train_X = train_X_tmp.iloc[:].values
train_y = train.y.values

## 訓練データで訓練

In [6]:
XGB = XGBRegressor()

In [7]:
XGB = XGB.fit(train_X, train_y)

[14:34:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [8]:
XGB.feature_importances_

array([0.15236025, 0.15385875, 0.47403893, 0.02463321, 0.0714016 ,
       0.01714016, 0.01123533, 0.02024692, 0.0084546 , 0.06663023],
      dtype=float32)

In [9]:
sorted(
    zip(map(lambda x: round(x, 3), XGB.feature_importances_), train.iloc[:, [1, 2, 3, 4, 5, 6, 8, 9, 10, 11]].columns),
    reverse=True)

[(0.474, 'bedrooms'),
 (0.154, 'bathrooms'),
 (0.152, 'accommodates'),
 (0.071, 'host_response_rate'),
 (0.067, 'city_flag'),
 (0.025, 'beds'),
 (0.02, 'property_type_num'),
 (0.017, 'number_of_reviews'),
 (0.011, 'bed_type_flag'),
 (0.008, 'cancel_policy_flag')]

## テストデータで実行

In [10]:
# ターゲットと特徴量の分割
test_X_tmp = test.copy()
test_X_tmp.drop(['id', 'y'], axis=1, inplace=True)
test_x = test_X_tmp.iloc[:].values
test_y = test.y.values

In [11]:
test_x.shape, test_y.shape

((11065, 10), (11065,))

In [12]:
pred_y = XGB.predict(test_x)

In [13]:
np.sqrt(mean_squared_error(test_y, pred_y))

131.15672284114572

## 検証データで実行

In [ ]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk3')

In [ ]:
valid.shape

In [ ]:
# ID の保存
valid_pass = valid.id.values

In [ ]:
valid_X = valid.iloc[:, 1:].values

In [ ]:
valid_X.shape, train_X.shape

In [ ]:
# valid_X.head()

In [ ]:
pred_valid_y = XGB.predict(valid_X)

In [ ]:
pred_valid_y.shape

In [ ]:
type(valid_pass), type(pred_valid_y)

In [ ]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [ ]:
result_df.to_csv("./XGB_8.csv", header=False)